# Bankruptcy Prediction via Regression


## Dataset and problem




Source: Malcolm J. Beynon, Michael J. Peel, Variable precision rough set theory and data discretisation: an application to corporate failure prediction

1) Size

a. Sales

2) Profit

a. ROCE: profit before tax=capital employed (%)

b. FFTL: funds flow (earnings before interest, tax & depreciation)=total liabilities

3) Gearing

a. GEAR: (current liabilities + long-term debt)=total assets

b. CLTA: current liabilities=total assets

4) Liquidity

a. CACL: current assets=current liabilities

b. QACL: (current assets – stock)=current liabilities

c. WCTA: (current assets – current liabilities)=total assets

5) LAG: number of days between account year end and the date the annual report and accounts were filed at company registry.

6) AGE: number of years the company has been operating since incorporation date.

7) CHAUD: coded 1 if changed auditor in previous three years, 0 otherwise

8) BIG6: coded 1 if company auditor is a Big6 auditor, 0 otherwise

The target variable is FAIL, either = 1 or 0.




Write your own logistic regression code to classify companies using test data, compute the accuracy, confusion matrix, precision, recall. Start with writing individual functions, then the main code that calls the functions above to train a model using the training data and return theta, then test the model using the test data, compute accuracy, confusion matrix, precision, recall


All the imports combined in one

In [ ]:
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

import pandas as pd


import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from random import seed

seed(1)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest




import cvxopt as opt
from cvxopt import matrix, solvers


In [ ]:

bankruptcy_df = pd.read_csv(r'bankruptcy.csv',header =0) # '/content/gdrive/My Drive/models/bankruptcy/bankruptcy.csv' if using Google Drive, also uncomment the code for drive mounting
bankruptcy_df.head()



,Firm,SALES,ROCE,FFTL,GEAR,CLTA,CACL,QACL,WCTA,LAG,AGE,CHAUD,BIG6,FAIL
0,o1,6762,7.5364,0.1545,0.6233,0.6233,1.5489,0.7356,0.3422,96,74,0,0,0
1,o2,16149,-1.0712,0.0271,1.2218,1.2218,0.6236,0.3153,-0.4599,287,29,0,1,0
2,o3,8086,15.2024,0.6163,0.3307,0.3307,2.3553,1.7513,0.4482,64,51,0,1,0
3,o4,7646,31.2239,0.6312,0.5205,0.4829,1.6397,1.4935,0.3089,286,25,0,0,0
4,o5,36067,10.9613,0.3540,0.3786,0.3786,1.5852,1.1626,0.2216,301,33,0,1,0


## Data Exploration

Exploring the dataset



Take a look at the description of the data.


In [ ]:
bankruptcy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Firm    60 non-null     object 
 1   SALES   60 non-null     int64  
 2   ROCE    60 non-null     float64
 3   FFTL    60 non-null     float64
 4   GEAR    60 non-null     float64
 5   CLTA    60 non-null     float64
 6   CACL    60 non-null     float64
 7   QACL    60 non-null     float64
 8   WCTA    60 non-null     float64
 9   LAG     60 non-null     int64  
 10  AGE     60 non-null     int64  
 11  CHAUD   60 non-null     int64  
 12  BIG6    60 non-null     int64  
 13  FAIL    60 non-null     int64  
dtypes: float64(7), int64(6), object(1)
memory usage: 6.7+ KB




Class distribution is balanced so doesn't need to resample


In [ ]:
bankruptcy_df[bankruptcy_df['FAIL'] ==0] = -1
bankruptcy_df.groupby(['FAIL'])[['FAIL']].count()

,FAIL
FAIL,
-1,30
1,30




Now get a summary of the numerical attributes


In [ ]:
bankruptcy_df.describe()

,SALES,ROCE,FFTL,GEAR,CLTA,CACL,QACL,WCTA,LAG,AGE,CHAUD,BIG6,FAIL
count,60.00000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,5673.00000,-1.837925,-0.465092,-0.081783,-0.148318,0.005928,-0.163737,-0.519848,134.066667,9.916667,-0.350000,-0.316667,0.000000
std,10402.93709,7.888941,0.547884,0.937403,0.873916,1.045462,0.874284,0.509825,144.244480,18.584021,0.732421,0.770025,1.008439
min,-1.00000,-31.254000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.00000,-1.708625,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,1429.00000,-1.000000,-0.664150,-0.305100,-0.350700,-0.251300,-0.357650,-0.873500,42.500000,0.500000,-0.500000,-0.500000,0.000000
75%,5780.00000,-1.000000,0.064650,0.803050,0.701225,0.925075,0.559350,-0.045675,295.000000,14.000000,0.000000,0.000000,1.000000
max,48162.00000,26.500600,0.427700,1.486500,1.486500,2.067400,1.849300,0.408400,393.000000,90.000000,1.000000,1.000000,1.000000


## Data Preparation

Dividing the dataset into training and test datasets is the very first thing to do. The test dataset should not be touched, but only be used to see how the model performs after all, and afterward do not adjust the model further. The training dataset is used for training (which actually further divided, and used to create validation and training datasets during grid search)

In [ ]:

X = bankruptcy_df[bankruptcy_df.columns.difference(['Firm', 'FAIL'])]
y = bankruptcy_df[['FAIL']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12346789)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)





Select k features only using univariate feature selection by selecting the best features based on univariate statistical tests. Note that the test data needs to be applied the same transform that was used for training (in this case we would select the same features as we did for the traing set)


In [ ]:
k = 10
prep = SelectKBest(k=k)
X_train = prep.fit_transform(X_train, y_train.values.flatten())
X_test = prep.fit_transform(X_test, y_test.values.flatten())

## Modeling




The following is a SVM model from scratch


### As per Cuong Do,  sigma is $\sigma$ for Gaussian Kernel OR $d$ in the case of Polynomial Kernel. type chooses between the 3 type of kernel and based on all information .. Type 2 is linear while the others are unknown so going to assume that type 0 is Gaussian and Type 1 is Polynomial

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
def LinearKernel(XTest, XTrain, extra=None):

    # kernel_mat = np.zeros(XTest.shape[0],XTest.shape[0])
    # for i,rows in enumerate(XTest):
    #     for j,columns in enumerate(XTrain.T):
    #         kernel_mat[i,j] = np.dot(rows, columns)
    # return kernel_mat


    return XTest @ XTrain.T

def PolyKernel(XTest, XTrain, d=0):


    # kernel_mat = np.zeros((XTest.shape[0],XTest.shape[0]))
    # for i,rows in enumerate(XTest): #rows here represnets x_(i)
    #     for j,columns in enumerate(XTrain): # column here atfer transpose represents x_(j)
    #         kernel_mat[i,j] = (np.dot(rows,columns) + 1) ** d
    #  return kernel_mat

    kernel_mat = (XTest @ XTrain.T + 1) ** d

    return kernel_mat

def GaussianKernel(XTest, XTrain, sigma=0):


    # kernel_mat = np.zeros((XTest.shape[0],XTest.shape[0]))
    # for i,rows in enumerate(XTest):
    #     for j,columns in enumerate(XTrain):
    #         kernel_mat[i,j] = np.exp(-np.sqrt(np.sum((rows - columns) ** 2)) / (2 * sigma **2))

    # return kernel_mat

    # ||x_i - x_j ||^ 2 is equivalent to (x_i . x_i - 2 x_i . x_j + x_j . x_j ) ** 0.5
    # see: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html

    kernel_mat = np.exp(-euclidean_distances(XTest,XTrain) / (2* sigma **2))
    return kernel_mat



#Create Function pointers
kernel_dict = {
    0: GaussianKernel,
    1: PolyKernel,
    2: LinearKernel,
}


def kernel(XTest, XTrain, kernel_type=0, extra=0):
    # Write your code below
    if  kernel_type != 2 and extra == 0:
        raise ValueError("d / sigma is going to be 0 will lead to value of 1 or divisible by 0 respectively")

    return kernel_dict[kernel_type](XTest, XTrain, extra)

In [ ]:
def intercept(XTest, XTrain, yTrain, alpha):
    # Write your code below

    #assume Linear Kernel
    K = LinearKernel(XTrain,XTrain,None)

    #mask out alpha_i < 0
    mask = alpha > 0

    mask = mask.flatten()
    alpha_actual = alpha[mask]
    N_s = alpha_actual.shape[0]

    idxs = np.arange(len(alpha))[mask] # indexs for x which are going to act as support vectors
    newY = yTrain[mask]

    b = 0.0
    print("Number of Support Vecs", N_s)
    for i in range(N_s):
        b += newY[i]
        for j in range(N_s):
            b -= alpha_actual[i] * newY[i] * K[idxs[i],idxs[j]]
    b/=N_s
    print(b)

    return b

In [ ]:
def predict(XTest, XTrain, yTrain, alpha):
    # Write your code below
    b = intercept(XTest,XTrain,yTrain,alpha)

    answer = np.zeros(XTest.shape[0])
    print(XTest.shape)
    print(alpha.shape)
    for i in range(XTest.shape[0]):
        for j in range(XTest.shape[1]):
            # answer[i] += alpha[j] * yTrain[j] *
            pass


    # pred = np.sign(answer)
    return None


In [ ]:
sigma = 2
kernel_type = 2



# Linear Kernel
TrainSize = X_train.shape[0]
K = kernel(X_train, X_train, kernel_type, sigma)
a0 = np.random.randn(TrainSize)
# Inequality that individual alpha>=0
G = matrix(np.eye(TrainSize))
h = matrix(np.zeros(TrainSize))

# Equality that sum(alpha_i*y_i)=0
A = opt.matrix(np.double(y_train))
b = matrix(0.0)

# Change from min to max optimization by multiplying with -1
# Regularization term to force H positive definite
ymat = np.diag(y_train.values.flatten())
P = 0.5 * np.dot(ymat, np.dot(K, ymat)) + 1e-5 * np.identity(TrainSize)
q = opt.matrix(-np.ones((TrainSize,1)))

opts = {'maxiters':100000}
solvers.options['show_progress'] = False
sol = solvers.qp(matrix(P), q, G, h, A.T, b, initvals = a0, options=opts) # solvers.qp to solve the optimization
alpha = np.array(sol['x'])


pred = predict(X_test, X_train, y_train.values, alpha)
# print(pred.shape,y_test.shape)
print(y_test.values.flatten())
# print('Accuracy: %f\n' % (np.mean(pred == y_test)*100))

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00  0.0000e+00  4e+01  6e+00  0e+00
 1:  0.0000e+00 -5.5511e-17  4e-01  6e-02  1e-16
 2: -1.6759e-17  0.0000e+00  4e-03  6e-04  1e-16
 3: -3.5178e-19 -6.7763e-21  4e-05  6e-06  1e-16
 4: -1.0521e-21  2.6470e-22  4e-07  6e-08  4e-17
 5: -1.3812e-23  0.0000e+00  4e-09  6e-10  4e-17
Optimal solution found.
Number of Support Vecs 33
[-0.27272727]
(18, 10)
(42, 1)
[ 1 -1  1 -1 -1 -1 -1  1 -1  1  1  1 -1  1 -1  1  1 -1]
